In [1]:
import numpy as np
import pandas as pd
from glob import glob
from stopsignalmetrics import StopData, SSRTmodel

In [2]:
def read_cond_file(filey):
    cond_df = pd.read_excel(filey)
    cond_df = cond_df.replace(r'^\s*$', np.nan, regex=True).replace('?', np.nan)  
    return cond_df

In [9]:
#these are the 339 subjects who passed the criteria discussed in the manuscriptinfo
full_passed_turkers = [4, 5, 9, 11, 12, 13, 15, 16, 17, 19, 20, 23, 25, 28, 30, 31, 32,
                       33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51,
                       52, 53, 54, 55, 56, 58, 59, 60, 62, 63, 64, 66, 71, 74, 76, 80,
                       81, 82, 85, 87, 88, 89, 90, 91, 92, 94, 96, 97, 99, 100, 101,
                       102, 103, 105, 106, 107, 108, 109, 112, 113, 114, 117, 122, 123,
                       124, 125, 126, 127, 128, 132, 133, 134, 136, 139, 140, 141, 142,
                       144, 146, 150, 151, 154, 156, 158, 160, 161, 162, 165, 167, 168,
                       171, 172, 173, 175, 177, 179, 180, 183, 184, 185, 186, 188, 189,
                       190, 193, 195, 198, 199, 201, 202, 203, 205, 207, 208, 210, 212,
                       214, 215, 216, 217, 219, 220, 221, 222, 223, 226, 227, 228, 230,
                       232, 233, 234, 235, 236, 237, 239, 240, 241, 243, 244, 245, 246,
                       247, 251, 252, 254, 256, 257, 259, 260, 266, 268, 270, 271, 273,
                       274, 276, 278, 280, 282, 283, 284, 285, 288, 292, 293, 294, 295,
                       296, 297, 298, 299, 301, 302, 304, 305, 307, 309, 311, 313, 314,
                       317, 323, 324, 325, 327, 328, 330, 333, 334, 335, 336, 338, 340,
                       342, 343, 345, 346, 349, 350, 354, 355, 357, 358, 360, 362, 363,
                       364, 365, 368, 369, 375, 376, 377, 383, 384, 386, 387, 388, 389,
                       390, 394, 396, 397, 398, 399, 400, 403, 405, 406, 407, 409, 410,
                       411, 412, 414, 415, 416, 418, 419, 420, 421, 422, 423, 425, 426,
                       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 438, 439, 441,
                       443, 444, 445, 446, 448, 449, 450, 451, 454, 456, 457, 458, 459,
                       460, 461, 463, 465, 466, 467, 469, 470, 473, 474, 476, 477, 479,
                       480, 481, 482, 483, 486, 489, 492, 493, 494, 495, 496, 497, 499,
                       501, 504, 505, 507, 508, 509, 512, 514, 515, 516, 520, 521, 522,
                       523, 526, 527, 529, 533, 535, 539, 543, 545, 546, 548, 549, 550,
                       559, 560]

In [13]:
default_map = {
        'columns': {'ID': 'Subject',
                    'block': 'Block',
                    'condition': 'TrialType',
                    'SSD': 'StopSignalDelay',
                    'goRT': 'GoRT',
                    'stopRT': 'StopFailureRT',
                    'response': 'GoTrialResponse',
                    'correct_response': 'GoTrialCorrectResponse',
                   },
        'key_codes': {'go': 'go',
                      'stop': 'stop',
                      'correct': 1,
                      'incorrect': 0,
                      'noResponse': 0,
                     }
    }
mapping_dicts = {
    'Matzke': {
        'columns': {'ID': 'Subject',
                    'condition': 'SS',
                    'SSD': 'StopSignalDelay',
                    'goRT': 'Target.RT',
                    'stopRT': 'TargetDelay.RT',
                    'response': 'R',
                    'correct_response': 'S',
                   },
        'key_codes': {'go': 'go',
                      'stop': 'stop',
                      'correct': 1,
                      'incorrect': 0,
                      'noResponse': 0,
                     }
    },
    'Saccades': {
        'columns': {'ID': 'Subject',
                    'block': 'Block',
                    'condition': 'TrialType',
                    'SSD': 'StopSignalDelay',
                    'goRT': 'GoRT',
                    'stopRT': 'StopFailureRT',
                    'response': 'GoTrialResponse',
                    'correct_response': 'GoTrialCorrectResponse',
                   },
        'key_codes': {'go': 'go',
                      'stop': 'Stop',
                      'correct': 1,
                      'incorrect': 0,
                      'noResponse': 0,
                     }
    }

}

In [32]:
sd_dict = {}
ssrt_df_dict = {}
for filey in [f for f in glob('normative_stopping_data/*') if '~$' not in f]:
    condition_name = filey.split('/Data')[-1].replace('.xlsx', '')
    curr_df = read_cond_file(filey)
    if 'Turk' in condition_name:
        curr_df = curr_df[curr_df.Subject.isin(full_passed_turkers)].copy()
    if 'Matzke' in condition_name:
        del curr_df['SSD']
    clean_data = StopData(mapping_dicts.get(condition_name, default_map)).fit_transform(curr_df)
    clean_data = clean_data[(clean_data.SSD >= 0) | (clean_data.SSD.isnull())].copy()  # ignore negative SSDs
    ssrt_metrics = SSRTmodel(model='replacement').fit_transform(clean_data, level='group')
    ssrt_df_dict[condition_name] = ssrt_metrics
    sd_dict[condition_name] = ssrt_metrics['SSRT'].std()
sd_dict

{'Deadline1500ms': 34.720568393562615,
 'Saccades': 34.211348598704376,
 'BtwnSubjVisual2': 29.775339862087897,
 'FixedSSDs2': 55.90494002179574,
 'BtwnSubjAuditory2': 35.76913227708826,
 'Deadline2700ms': 56.488884882220276,
 'TurkSimpleHigh': 44.977463388777046,
 'Deadline1300ms': 27.6090681978959,
 'WithinSubjCentralGoSimple': 36.33724505817492,
 'WithinSubjPeriphGoSimple': 30.4702282005005,
 'Matzke': 84.85650409521057,
 'TurkSimpleLow': 53.42240767811976,
 'Deadline1700ms': 39.60555763489168,
 'Deadline2300ms': 53.48093612753764,
 'BtwnSubjAuditory1': 35.26281790159863,
 'BtwnSubjVisual1': 31.693386222585243,
 'FixedSSDs1': 34.55797481318032,
 'Deadline2500ms': 56.45605367392923,
 'StopProbabilityLow': 37.0234708800525,
 'StopProbabilityHigh': 41.95826722296842}

In [33]:
sd_df = pd.DataFrame(sd_dict, index=['SD of SSRT']).T
sd_df

,SD of SSRT
Deadline1500ms,34.720568
Saccades,34.211349
BtwnSubjVisual2,29.775340
FixedSSDs2,55.904940
BtwnSubjAuditory2,35.769132
Deadline2700ms,56.488885
TurkSimpleHigh,44.977463
Deadline1300ms,27.609068
WithinSubjCentralGoSimple,36.337245
WithinSubjPeriphGoSimple,30.470228


In [34]:
sd_df.describe()

,SD of SSRT
count,20.000000
mean,42.729080
std,13.804978
min,27.609068
25%,34.471318
50%,36.680358
75%,53.437040
max,84.856504
